# Feature Engineering and Modelling

---

1. Import packages
2. Load data
3. Modelling

---

## 1. Import packages

In [1]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import datetime
import matplotlib.pyplot as plt

# Shows plots in jupyter notebook
%matplotlib inline

# Set plot style
sns.set(color_codes=True)

---
## 2. Load data

In [3]:
df = pd.read_csv('./data_for_predictions.csv')
df.drop(columns=["Unnamed: 0"], inplace=True)
df.head()

,id,cons_12m,cons_gas_12m,cons_last_month,forecast_cons_12m,forecast_discount_energy,forecast_meter_rent_12m,forecast_price_energy_off_peak,forecast_price_energy_peak,forecast_price_pow_off_peak,...,months_modif_prod,months_renewal,channel_MISSING,channel_ewpakwlliwisiwduibdlfmalxowmwpci,channel_foosdfpfkusacimwkcsosbicdxkicaua,channel_lmkebamcaaclubfxadlmueccxoimlema,channel_usilxuppasemubllopkaafesmlibmsdf,origin_up_kamkkxfxxuwbdslkwifmmcsiusiuosws,origin_up_ldkssxwpmemidmecebumciepifcamkci,origin_up_lxidpiddsbxsbosboudacockeimpuepw
0,24011ae4ebbe3035111d65fa7c15bc57,0.000000,4.739944,0.000000,0.000000,0.0,0.444045,0.114481,0.098142,40.606701,...,2,6,0,0,1,0,0,0,0,1
1,d29c2c54acc38ff3c0614d0a653813dd,3.668479,0.000000,0.000000,2.280920,0.0,1.237292,0.145711,0.000000,44.311378,...,76,4,1,0,0,0,0,1,0,0
2,764c75f661154dac3a6c254cd082ea7d,2.736397,0.000000,0.000000,1.689841,0.0,1.599009,0.165794,0.087899,44.311378,...,68,8,0,0,1,0,0,1,0,0
3,bba03439a292a1e166f80264c16191cb,3.200029,0.000000,0.000000,2.382089,0.0,1.318689,0.146694,0.000000,44.311378,...,69,9,0,0,0,1,0,1,0,0
4,149d57cf92fc41cf94415803a877cb4b,3.646011,0.000000,2.721811,2.650065,0.0,2.122969,0.116900,0.100015,40.606701,...,71,9,1,0,0,0,0,1,0,0


---

## 3. Modelling

We now have a dataset containing features that we have engineered and we are ready to start training a predictive model. Remember, we only need to focus on training a `Random Forest` classifier.

In [4]:
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

### Data sampling

The first thing we want to do is split our dataset into training and test samples. The reason why we do this, is so that we can simulate a real life situation by generating predictions for our test sample, without showing the predictive model these data points. This gives us the ability to see how well our model is able to generalise to new data, which is critical.

A typical % to dedicate to testing is between 20-30, for this example we will use a 75-25% split between train and test respectively.

In [5]:
# Make a copy of our data
train_df = df.copy()

# Separate target variable from independent variables
y = df['churn']
X = df.drop(columns=['id', 'churn'])
print(X.shape)
print(y.shape)

(14606, 61)
(14606,)


In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(10954, 61)
(10954,)
(3652, 61)
(3652,)


### Model training

Once again, we are using a `Random Forest` classifier in this example. A Random Forest sits within the category of `ensemble` algorithms because internally the `Forest` refers to a collection of `Decision Trees` which are tree-based learning algorithms. As the data scientist, you can control how large the forest is (that is, how many decision trees you want to include).

The reason why an `ensemble` algorithm is powerful is because of the laws of averaging, weak learners and the central limit theorem. If we take a single decision tree and give it a sample of data and some parameters, it will learn patterns from the data. It may be overfit or it may be underfit, but that is now our only hope, that single algorithm. 

With `ensemble` methods, instead of banking on 1 single trained model, we can train 1000's of decision trees, all using different splits of the data and learning different patterns. It would be like asking 1000 people to all learn how to code. You would end up with 1000 people with different answers, methods and styles! The weak learner notion applies here too, it has been found that if you train your learners not to overfit, but to learn weak patterns within the data and you have a lot of these weak learners, together they come together to form a highly predictive pool of knowledge! This is a real life application of many brains are better than 1.

Now instead of relying on 1 single decision tree for prediction, the random forest puts it to the overall views of the entire collection of decision trees. Some ensemble algorithms using a voting approach to decide which prediction is best, others using averaging. 

As we increase the number of learners, the idea is that the random forest's performance should converge to its best possible solution.

Some additional advantages of the random forest classifier include:

- The random forest uses a rule-based approach instead of a distance calculation and so features do not need to be scaled
- It is able to handle non-linear parameters better than linear based models

On the flip side, some disadvantages of the random forest classifier include:

- The computational power needed to train a random forest on a large dataset is high, since we need to build a whole ensemble of estimators.
- Training time can be longer due to the increased complexity and size of thee ensemble

In [9]:
# Add model training in here!
model = RandomForestClassifier(
    n_estimators=1000,   
    max_depth=10,         
    random_state=42,     
    class_weight='balanced'
) # Add parameters to the model!
model.fit(X_train, y_train) # Complete this method call!

RandomForestClassifier(class_weight='balanced', max_depth=10, n_estimators=1000,
                       random_state=42)

### Evaluation

Now let's evaluate how well this trained model is able to predict the values of the test dataset.

In [10]:
# Generate predictions here!
y_pred = model.predict(X_test)

In [12]:
from sklearn.metrics import classification_report

In [13]:
# Calculate performance metrics here!
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.92      0.90      0.91      3286
           1       0.22      0.26      0.24       366

    accuracy                           0.83      3652
   macro avg       0.57      0.58      0.57      3652
weighted avg       0.85      0.83      0.84      3652



In [14]:
# 1. Calculate the correlation of every column in X_train with y_train
correlations = X_train.corrwith(y_train)

# 2. Create a clean DataFrame for display
# We take the absolute value as well because a strong negative correlation 
# (e.g., high tenure = low churn) is just as important as a positive one.
correlation_df = pd.DataFrame({
    'Feature': correlations.index,
    'Correlation': correlations.values,
    'Abs_Correlation': correlations.abs().values
})

# 3. Sort by the absolute impact to see the most influential features first
correlation_df = correlation_df.sort_values(by='Abs_Correlation', ascending=False).reset_index(drop=True)

# Display the result
print(correlation_df[['Feature', 'Correlation']])

                                       Feature  Correlation
0   origin_up_lxidpiddsbxsbosboudacockeimpuepw     0.103099
1                           margin_net_pow_ele     0.099263
2                         margin_gross_pow_ele     0.099202
3   origin_up_kamkkxfxxuwbdslkwifmmcsiusiuosws    -0.087103
4                                 months_activ    -0.084567
..                                         ...          ...
56                     var_year_price_peak_var     0.004235
57             offpeak_diff_dec_january_energy     0.003422
58                       var_6m_price_mid_peak     0.002045
59                   var_6m_price_mid_peak_fix     0.002045
60          peak_mid_peak_var_max_monthly_diff     0.000001

[61 rows x 2 columns]


In [15]:
# Extract importances from your trained model
importances = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': model.feature_importances_
}).sort_values(by='Importance', ascending=False)

print(importances.head(10))

                        Feature  Importance
12           margin_net_pow_ele    0.069680
11         margin_gross_pow_ele    0.068436
0                      cons_12m    0.050450
49                 months_activ    0.042733
5       forecast_meter_rent_12m    0.039984
2               cons_last_month    0.037827
14                   net_margin    0.035472
3             forecast_cons_12m    0.033404
51            months_modif_prod    0.032335
36  off_peak_peak_var_mean_diff    0.028336


In [16]:
# 1. Sort and select the top N features (e.g., top 10)
top_n = 10
top_features_list = importances.sort_values(by='Importance', ascending=False).head(top_n)['Feature'].tolist()

# 2. Alternatively, extract ALL features in order of importance
all_features_ordered = importances.sort_values(by='Importance', ascending=False)['Feature'].tolist()

print(top_features_list)

['margin_net_pow_ele', 'margin_gross_pow_ele', 'cons_12m', 'months_activ', 'forecast_meter_rent_12m', 'cons_last_month', 'net_margin', 'forecast_cons_12m', 'months_modif_prod', 'off_peak_peak_var_mean_diff']


In [20]:
model2 = RandomForestClassifier(
    n_estimators=1000,   
    max_depth=10,         
    random_state=42,     
    class_weight='balanced'
) # Add parameters to the model!
model2.fit(X_train[top_features_list], y_train) # Complete this method call!

RandomForestClassifier(class_weight='balanced', max_depth=10, n_estimators=1000,
                       random_state=42)

In [22]:
y_pred2 = model2.predict(X_test[top_features_list])

In [23]:
print(classification_report(y_test, y_pred2))

              precision    recall  f1-score   support

           0       0.92      0.88      0.90      3286
           1       0.20      0.28      0.24       366

    accuracy                           0.82      3652
   macro avg       0.56      0.58      0.57      3652
weighted avg       0.84      0.82      0.83      3652



In [24]:
from imblearn.over_sampling import SMOTE

In [25]:
# Create synthetic samples for the minority class
sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(X_train, y_train)

# Re-train the model on the balanced dataset
model.fit(X_res, y_res)

RandomForestClassifier(class_weight='balanced', max_depth=10, n_estimators=1000,
                       random_state=42)

In [26]:
# Instead of model.predict(), get the probabilities
probabilities = model.predict_proba(X_test)[:, 1]

# Set a lower threshold to catch more churners (increases Recall, may lower Precision)
custom_preds = (probabilities > 0.3).astype(int)

In [27]:
y_pred = model.predict(X_test)

In [28]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.91      0.93      0.92      3286
           1       0.22      0.18      0.20       366

    accuracy                           0.85      3652
   macro avg       0.56      0.55      0.56      3652
weighted avg       0.84      0.85      0.85      3652



In [35]:
model3 = RandomForestClassifier(
    n_estimators=1000,   
    max_depth=10,         
    random_state=42,     
    class_weight='balanced'
) # Add parameters to the model!
model3.fit(X_train[['months_activ','margin_net_pow_ele','cons_12m','forecast_meter_rent_12m']], y_train)

RandomForestClassifier(class_weight='balanced', max_depth=10, n_estimators=1000,
                       random_state=42)

In [36]:
y_pred = model3.predict(X_test[['months_activ','margin_net_pow_ele','cons_12m','forecast_meter_rent_12m']])

In [37]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.92      0.86      0.89      3286
           1       0.18      0.29      0.23       366

    accuracy                           0.80      3652
   macro avg       0.55      0.57      0.56      3652
weighted avg       0.84      0.80      0.82      3652



In [38]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [39]:
# Calculate VIF for each feature
vif_data = pd.DataFrame()
vif_data["feature"] = X_train.columns
vif_data["VIF"] = [variance_inflation_factor(X_train.values, i) for i in range(len(X_train.columns))]

print(vif_data.sort_values(by="VIF", ascending=False))

C:\Users\ephet\anaconda3\Lib\site-packages\statsmodels\stats\outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


                            feature       VIF
30        var_6m_price_mid_peak_fix       inf
19      var_year_price_off_peak_fix       inf
21      var_year_price_mid_peak_fix       inf
22          var_year_price_off_peak       inf
23              var_year_price_peak       inf
..                              ...       ...
34  offpeak_diff_dec_january_energy  3.606751
51                months_modif_prod  3.550876
14                       net_margin  3.315782
4          forecast_discount_energy  2.772548
35   offpeak_diff_dec_january_power  2.119590

[61 rows x 2 columns]


In [42]:
# Create a copy of the features to prune
X_basis = X_train.copy()

# Function to automatically prune the 'redundant' basis vectors
def reduce_vif(df):
    thresh = 5.0 # Standard threshold for 'independent' features
    for i in range(df.shape[1]):
        vif = [variance_inflation_factor(df.values, j) for j in range(df.shape[1])]
        max_vif = max(vif)
        if max_vif == float('inf') or max_vif > thresh:
            index_max_vif = vif.index(max_vif)
            print(f"Dropping '{df.columns[index_max_vif]}' (VIF: {max_vif})")
            df = df.drop(columns=[df.columns[index_max_vif]])
        else:
            break
    return df

X_final_basis = reduce_vif(X_basis)
print(f"\nRemaining 'Standard Basis' Columns: {len(X_final_basis.columns)}")

C:\Users\ephet\anaconda3\Lib\site-packages\statsmodels\stats\outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


Dropping 'var_year_price_off_peak_var' (VIF: inf)


C:\Users\ephet\anaconda3\Lib\site-packages\statsmodels\stats\outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


Dropping 'var_year_price_peak_var' (VIF: inf)


C:\Users\ephet\anaconda3\Lib\site-packages\statsmodels\stats\outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


...

Dropping 'var_6m_price_mid_peak_var' (VIF: inf)


C:\Users\ephet\anaconda3\Lib\site-packages\statsmodels\stats\outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


Dropping 'off_peak_peak_var_mean_diff' (VIF: inf)


C:\Users\ephet\anaconda3\Lib\site-packages\statsmodels\stats\outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


Dropping 'off_peak_peak_fix_mean_diff' (VIF: inf)
Dropping 'var_6m_price_mid_peak' (VIF: 78615374082.3841)
Dropping 'var_6m_price_off_peak_fix' (VIF: 52500826254.73437)
Dropping 'var_year_price_mid_peak' (VIF: 47399326703.12266)
Dropping 'var_year_price_peak' (VIF: 21044612223.55111)
Dropping 'var_year_price_off_peak' (VIF: 9825646097.744844)
Dropping 'var_6m_price_peak' (VIF: 7645802884.697521)
Dropping 'off_peak_mid_peak_fix_max_monthly_diff' (VIF: 127076.43837511417)
Dropping 'margin_gross_pow_ele' (VIF: 10968.81279852279)
Dropping 'off_peak_mid_peak_fix_mean_diff' (VIF: 2625.69429067543)
Dropping 'forecast_price_pow_off_peak' (VIF: 1733.0759985149157)
Dropping 'off_peak_peak_var_max_monthly_diff' (VIF: 1293.0925473866453)
Dropping 'peak_mid_peak_fix_max_monthly_diff' (VIF: 822.3310405003061)
Dropping 'off_peak_mid_peak_var_mean_diff' (VIF: 563.0177028584426)
Dropping 'off_peak_mid_peak_var_max_monthly_diff' (VIF: 379.4803769552165)
Dropping 'tenure' (VIF: 319.817106577573)
Dropping

In [40]:
from sklearn.feature_selection import RFE

In [43]:
X_final_basis

,cons_gas_12m,forecast_discount_energy,imp_cons,margin_net_pow_ele,net_margin,var_year_price_off_peak_fix,var_year_price_peak_fix,var_6m_price_mid_peak_fix,offpeak_diff_dec_january_energy,offpeak_diff_dec_january_power,...,peak_mid_peak_fix_mean_diff,months_to_end,months_modif_prod,months_renewal,channel_MISSING,channel_ewpakwlliwisiwduibdlfmalxowmwpci,channel_lmkebamcaaclubfxadlmueccxoimlema,channel_usilxuppasemubllopkaafesmlibmsdf,origin_up_kamkkxfxxuwbdslkwifmmcsiusiuosws,origin_up_ldkssxwpmemidmecebumciepifcamkci
11674,0.000000,0.0,2.045440,34.68,94.69,1.155970e-01,0.000000e+00,0.000000,0.001395,1.177778,...,0.000000,8,27,3,0,0,0,0,0,0
3728,0.000000,0.0,0.000000,31.64,40.29,6.464721e-03,0.000000e+00,0.000000,-0.004547,0.177779,...,0.000000,2,45,9,0,0,0,0,0,0
1041,3.759592,0.0,2.418218,25.92,435.09,0.000000e+00,0.000000e+00,0.000000,-0.008298,0.000000,...,8.145775,4,8,7,0,0,0,0,0,0
12195,0.000000,0.0,0.000000,19.56,69.73,2.101764e-01,0.000000e+00,0.000000,-0.001492,1.177778,...,0.000000,9,38,2,0,0,0,0,0,1
14334,0.000000,0.0,0.000000,15.72,90.37,7.661956e-03,0.000000e+00,0.000000,-0.003767,0.177779,...,0.000000,6,53,4,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5191,0.000000,0.0,0.000000,24.25,146.77,7.037428e-03,2.533545e-03,0.000708,-0.011850,0.162916,...,8.126769,6,6,5,0,0,0,1,1,0
13418,0.000000,0.0,2.049140,41.88,177.73,3.436364e-12,1.848000e-12,0.000000,-0.010140,0.000004,...,8.113194,0,0,11,1,0,0,0,0,1
5390,0.000000,0.0,2.020113,3.72,176.48,2.211738e-03,7.962682e-04,0.000708,-0.007016,0.162916,...,8.115909,10,76,1,1,0,0,0,1,0
860,0.000000,0.0,0.000000,22.52,66.75,8.619686e-03,0.000000e+00,0.000000,-0.003423,0.177779,...,0.000000,5,2,6,0,0,0,0,0,0


In [57]:
cols_to_drop = [
    'channel_MISSING', 
    'origin_up_kamkkxfxxuwbdslkwifmmcsiusiuosws', 
    'channel_ewpakwlliwisiwduibdlfmalxowmwpci',
    'channel_lmkebamcaaclubfxadlmueccxoimlema',
    'channel_usilxuppasemubllopkaafesmlibmsdf',
    'origin_up_ldkssxwpmemidmecebumciepifcamkci'
]

# Drop them and save to a new variable
X_cleaned = X_final_basis.drop(cols_to_drop, axis=1)

In [58]:
# 1. Identify the 'Basis' columns
basis_features = X_cleaned.columns.tolist()

# 2. Refit the model using only these columns
# We keep the 'balanced' weight to help with that low Recall score
model_basis = RandomForestClassifier(
    n_estimators=1000,
    max_depth=10,
    random_state=42,
    class_weight='balanced'
)

model_basis.fit(X_train[basis_features], y_train)

# 3. Predict and Evaluate
y_pred_basis = model_basis.predict(X_test[basis_features])
print(classification_report(y_test, y_pred_basis))

              precision    recall  f1-score   support

           0       0.92      0.89      0.90      3286
           1       0.23      0.28      0.25       366

    accuracy                           0.83      3652
   macro avg       0.57      0.59      0.58      3652
weighted avg       0.85      0.83      0.84      3652



Just looking at accuracy is going to give you a poor predictor because it is easy to accept simply failing to identify True Positives since their overall prediction success would be biased by the population disparity. I don't want to throw a ton of variables at the wall and claim that will work thanks to overfitting, so my goal was to find out which columns were the most explanatory in actually catching churn. I use the classification report to take a look at f1-scores and how it breaks down into precision and recall. For reasons I have just explained we want to pay more attention to recall. To cut down on multicollinearity I used a recursive culling algorithm based on its variance inflation factor (highest VIF/infinite VIF were dropped), removing all redundant features (using a standard threshold of 5.0). With this list of 21 columns we have the bases of a "prediction space". From this list I removed those features which I assessed to be erroneus to predict Churn, then used RFE to find the 15 most explanatory features and pruned the rest from the model.

In [78]:
from sklearn.feature_selection import RFE
from sklearn.metrics import f1_score

In [65]:
# 1. Initialize the RFE selector
# We use the model_basis (Random Forest) as the 'judge'
rfe_selector = RFE(estimator=model_basis, n_features_to_select=15, step=1)

# 2. Fit RFE on the VIF-cleaned basis
rfe_selector.fit(X_train[basis_features], y_train)

# 3. Extract the final 'Basis' list
rfe_basis = X_train[basis_features].columns[rfe_selector.support_].tolist()

# 4. Train the final model on these 15 features
model_rfe = RandomForestClassifier(
    n_estimators=1000,
    max_depth=10,
    random_state=42,
    class_weight='balanced'
)
model_rfe.fit(X_train[rfe_basis], y_train)

# 5. Generate Predictions
y_pred_rfe = model_rfe.predict(X_test[rfe_basis])

# Print the final report
print(classification_report(y_test, y_pred_rfe))

              precision    recall  f1-score   support

           0       0.92      0.89      0.90      3286
           1       0.23      0.28      0.25       366

    accuracy                           0.83      3652
   macro avg       0.57      0.59      0.58      3652
weighted avg       0.85      0.83      0.84      3652



In [66]:
print(rfe_basis)

['cons_gas_12m', 'forecast_discount_energy', 'imp_cons', 'margin_net_pow_ele', 'net_margin', 'var_year_price_off_peak_fix', 'var_year_price_peak_fix', 'var_6m_price_mid_peak_fix', 'offpeak_diff_dec_january_energy', 'offpeak_diff_dec_january_power', 'peak_mid_peak_var_mean_diff', 'peak_mid_peak_fix_mean_diff', 'months_to_end', 'months_modif_prod', 'months_renewal']


I have continued I think that the model performance leaves a lot to be desired, the low precision suggests an endogeneity issue, where other primary drivers of churn are not incorporated in the dataset. I would say there is a lack of customer service data that could play a large explanatory role that we are missing in our current model. Nevertheless I continued, deciding to 

In [77]:
# 1. Initialize the RFE selector
# We use the model_basis (Random Forest) as the 'judge'
rfe_selector = RFE(estimator=model_basis, n_features_to_select=15, step=1)

# 2. Fit RFE on the VIF-cleaned basis
rfe_selector.fit(X_train[basis_features], y_train)

# 3. Extract the final 'Basis' list
rfe_basis = X_train[basis_features].columns[rfe_selector.support_].tolist()

# 4. Train the final model on these 15 features
model_rfe = RandomForestClassifier(
    n_estimators=1000,
    max_depth=10,
    random_state=42,
    class_weight='balanced'
)
model_rfe.fit(X_train[rfe_basis], y_train)

# 5. Generate Predictions
y_probs_rfe = model_rfe.predict_proba(X_test[rfe_basis])[:, 1]
thresholds = np.linspace(0, 1, 100)
macro_f1_scores = []

for t in thresholds:
    y_pred_t = (y_probs_rfe >= t).astype(int)
    # 'macro' calculates f1 for each label, and finds their unweighted mean
    score = f1_score(y_test, y_pred_t, average='macro')
    macro_f1_scores.append(score)

# 3. Find the threshold that maximizes this average
best_threshold_macro = thresholds[np.argmax(macro_f1_scores)]
best_macro_f1 = max(macro_f1_scores)

print(f"Optimal Threshold for Macro-F1: {best_threshold_macro:.4f}")

# 4. Generate final predictions
y_pred_macro = (y_probs >= best_threshold_macro).astype(int)
print(classification_report(y_test, y_pred_macro))

Optimal Threshold for Macro-F1: 0.5354
              precision    recall  f1-score   support

           0       0.92      0.93      0.92      3286
           1       0.28      0.23      0.25       366

    accuracy                           0.86      3652
   macro avg       0.60      0.58      0.59      3652
weighted avg       0.85      0.86      0.86      3652



In [79]:
# --- STAGE 1: Feature Selection & Model Training ---
rfe_selector = RFE(estimator=model_basis, n_features_to_select=15, step=1)
rfe_selector.fit(X_train[basis_features], y_train)
rfe_basis = X_train[basis_features].columns[rfe_selector.support_].tolist()

model_rfe = RandomForestClassifier(
    n_estimators=1000, 
    max_depth=10, 
    random_state=42, 
    class_weight='balanced'
)
model_rfe.fit(X_train[rfe_basis], y_train)

# --- STAGE 2: Probability Generation & Threshold Search ---
y_probs = model_rfe.predict_proba(X_test[rfe_basis])[:, 1]
thresholds = np.linspace(0, 1, 100)
macro_f1_scores = [f1_score(y_test, (y_probs >= t).astype(int), average='macro') for t in thresholds]

# Finding the "Narrow Net" (The Balanced/Conservative Threshold)
narrow_net_threshold = thresholds[np.argmax(macro_f1_scores)]

# Defining the "Big Net" (The High-Recall Screening Threshold)
# We set this lower to catch the "rare disease" candidates early
big_net_threshold = 0.30 

# --- STAGE 3: Implementing the Two-Tiered Manifestation ---
# Tier 1: Flagged by Big Net (Potential Churners)
y_pred_big = (y_probs >= big_net_threshold).astype(int)

# Tier 2: Flagged by Narrow Net (Confirmed/High-Conviction Churners)
y_pred_narrow = (y_probs >= narrow_net_threshold).astype(int)

print(f"--- BIG NET (Threshold: {big_net_threshold:.2f}) ---")
print(classification_report(y_test, y_pred_big))

print(f"--- NARROW NET (Threshold: {narrow_net_threshold:.4f}) ---")
print(classification_report(y_test, y_pred_narrow))

--- BIG NET (Threshold: 0.30) ---
              precision    recall  f1-score   support

           0       0.93      0.31      0.47      3286
           1       0.11      0.78      0.20       366

    accuracy                           0.36      3652
   macro avg       0.52      0.55      0.33      3652
weighted avg       0.85      0.36      0.44      3652

--- NARROW NET (Threshold: 0.5354) ---
              precision    recall  f1-score   support

           0       0.92      0.93      0.92      3286
           1       0.28      0.23      0.25       366

    accuracy                           0.86      3652
   macro avg       0.60      0.58      0.59      3652
weighted avg       0.85      0.86      0.86      3652



In [80]:
# 1. Get raw probabilities
y_probs = model_rfe.predict_proba(X_test[rfe_basis])[:, 1]
thresholds = np.linspace(0, 1, 100)

# --- STAGE 1: Optimize Big Net for Churner F1 (Class 1) ---
# We calculate F1 specifically for the minority class (pos_label=1)
f1_churners = [f1_score(y_test, (y_probs >= t).astype(int), pos_label=1) for t in thresholds]
big_net_threshold = thresholds[np.argmax(f1_churners)]

# --- STAGE 2: Optimize Narrow Net for Macro-F1 (Balance) ---
macro_f1_scores = [f1_score(y_test, (y_probs >= t).astype(int), average='macro') for t in thresholds]
narrow_net_threshold = thresholds[np.argmax(macro_f1_scores)]

# --- FINAL CATEGORIZATION ---
y_pred_big = (y_probs >= big_net_threshold).astype(int)
y_pred_narrow = (y_probs >= narrow_net_threshold).astype(int)

print(f"Dynamic Big Net Threshold (Max Churn F1): {big_net_threshold:.4f}")
print(f"Dynamic Narrow Net Threshold (Max Macro F1): {narrow_net_threshold:.4f}")

print("\n" + "="*30 + "\nBIG NET RESULTS\n" + "="*30)
print(classification_report(y_test, y_pred_big))

print("\n" + "="*30 + "\nNARROW NET RESULTS\n" + "="*30)
print(classification_report(y_test, y_pred_narrow))

Dynamic Big Net Threshold (Max Churn F1): 0.4848
Dynamic Narrow Net Threshold (Max Macro F1): 0.5354

BIG NET RESULTS
              precision    recall  f1-score   support

           0       0.92      0.87      0.89      3286
           1       0.21      0.32      0.25       366

    accuracy                           0.81      3652
   macro avg       0.57      0.59      0.57      3652
weighted avg       0.85      0.81      0.83      3652


NARROW NET RESULTS
              precision    recall  f1-score   support

           0       0.92      0.93      0.92      3286
           1       0.28      0.23      0.25       366

    accuracy                           0.86      3652
   macro avg       0.60      0.58      0.59      3652
weighted avg       0.85      0.86      0.86      3652



I chose to implement a "second opinion" framerwork. After a proof of concept trial using the arbitrary threshold of .3, I then expand to a two stage optimization strategy. The first filter optimizes for class 1 f1-score (churners), acting as a bigger net, followed by a narrow net, which optimizes for the average of the f-1 scores. This tiered architecture allows for a graduated business response: early-stage, low-investment interventions for the 'Big Net' pool, and rigorous, high-value retention measures for those who trigger the 'Narrow Net' second opinion.